# Atelier 3: Utilisation d'un serveur MCP pour interagir avec Google Agenda

Dans ce notebook, nous allons:
1. Créer un serveur MCP simple pour Google Agenda
2. Configurer l'authentification OAuth pour Google Agenda
3. Implémenter des outils pour créer et lister des événements
4. Intégrer ces outils avec un modèle de langage pour créer un assistant de planification

In [ ]:
import os
import json
import datetime
import requests
from IPython.display import Markdown, display, HTML
import pickle
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

## 1. Configuration de l'authentification Google

Pour utiliser l'API Google Agenda, nous devons configurer l'authentification OAuth. Voici les étapes détaillées:

### 1.1 Créer un projet dans la Console Google Cloud

1. Accédez à la [Console Google Cloud](https://console.cloud.google.com/)
2. Connectez-vous avec votre compte Google
3. En haut de la page, cliquez sur la liste déroulante des projets, puis sur "Nouveau projet"
4. Donnez un nom à votre projet (par exemple "Assistant Agenda")
5. Cliquez sur "Créer"
6. Attendez que le projet soit créé, puis sélectionnez-le dans la liste déroulante des projets

### 1.2 Activer l'API Google Calendar

1. Dans le menu de navigation à gauche, cliquez sur "APIs et services" > "Bibliothèque"
2. Dans la barre de recherche, tapez "Google Calendar API"
3. Cliquez sur "Google Calendar API" dans les résultats
4. Cliquez sur le bouton "Activer"
5. Attendez que l'API soit activée

### 1.3 Configurer l'écran de consentement OAuth

1. Dans le menu de navigation à gauche, cliquez sur "APIs et services" > "Écran de consentement OAuth"
2. Remplissez les informations obligatoires:
   - Nom de l'application: "Assistant Agenda"
   - Email de support utilisateur: votre adresse email
3. Sélectionnez "Externe" comme Cible
4. Poursuivez jusqu'à Créer

### 1.4 Créer des identifiants OAuth

1. Dans le menu de navigation à gauche, cliquez sur "APIs et services" > "Identifiants"
2. Cliquez sur "Créer des identifiants" > "ID client OAuth"
3. Sélectionnez "Application de bureau" comme type d'application
4. Donnez un nom à votre client OAuth (par exemple "Client Assistant Agenda")
5. Cliquez sur "Créer"
6. Une fenêtre apparaît avec votre ID client et votre clé secrète. Cliquez sur "Télécharger JSON"
7. Renommez le fichier téléchargé en `credentials.json`
8. Placez ce fichier dans le même répertoire que ce notebook
9. Accédez à "APIs et services" > "Écran de consentement OAuth"
3. Assurez-vous d'avoir ajouté votre adresse email comme utilisateur test dans "Audience"
4. Vérifiez que l'étendue `https://www.googleapis.com/auth/calendar` est bien configurée dans "Accès aux données"
5. Supprimez le fichier `token.pickle` s'il existe et réessayez l'authentification

### 1.5 Première authentification

Lors de la première exécution du code d'authentification:
1. Une fenêtre de navigateur s'ouvrira automatiquement
2. Connectez-vous avec votre compte Google
3. Vous verrez un avertissement indiquant que l'application n'est pas vérifiée
4. Cliquez sur "Continuer" pour autoriser l'accès à votre agenda
5. Une fois l'autorisation accordée, vous pouvez fermer la fenêtre du navigateur
6. Le code générera un fichier `token.pickle` qui stockera vos identifiants pour les utilisations futures

> **Note**: Si vous préférez ne pas configurer l'authentification réelle pour cet atelier, vous pouvez utiliser le mode simulation en laissant `USE_MOCK_CALENDAR = True` dans la cellule suivante.

In [ ]:
# Configuration de l'authentification Google

# Si vous préférez ne pas utiliser l'authentification réelle pour cet atelier,
# mettez USE_MOCK_CALENDAR à True pour simuler les opérations de calendrier
USE_MOCK_CALENDAR = False  # Mettre à True pour utiliser le mode simulation

# Portées nécessaires pour l'API Google Calendar
# Cette portée permet un accès complet à tous les calendriers de l'utilisateur
SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_calendar_service():
    """Obtient un service Google Calendar authentifié."""
    if USE_MOCK_CALENDAR:
        print("Utilisation du calendrier simulé (mode démo)")
        return MockCalendarService()
    
    creds = None
    # Le fichier token.pickle stocke les tokens d'accès et de rafraîchissement de l'utilisateur
    if os.path.exists('token.pickle'):
        print("Chargement des identifiants depuis token.pickle...")
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # Si les identifiants n'existent pas ou ne sont plus valides
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Rafraîchissement du token expiré...")
            creds.refresh(Request())
        else:
            if not os.path.exists('credentials.json'):
                raise FileNotFoundError("""
                Le fichier credentials.json est introuvable. 
                
                Veuillez suivre les instructions pour:
                1. Créer un projet dans la Console Google Cloud
                2. Activer l'API Google Calendar
                3. Configurer l'écran de consentement OAuth
                4. Créer des identifiants OAuth
                5. Télécharger le fichier JSON et le renommer en credentials.json
                """)
            
            print("Démarrage du flux d'authentification OAuth...")
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            print("Une fenêtre de navigateur va s'ouvrir pour l'authentification...")
            creds = flow.run_local_server(port=0)
            print("Authentification réussie!")
        
        # Sauvegarder les identifiants pour la prochaine exécution
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
            print("Identifiants sauvegardés dans token.pickle")
    
    service = build('calendar', 'v3', credentials=creds)
    print("Service Google Calendar authentifié avec succès")
    return service

# Classe pour simuler le service Calendar (pour la démo sans authentification)
class MockCalendarService:
    """Simule un service Google Calendar pour la démonstration."""
    def __init__(self):
        self.events = []
        self.next_id = 1
        print("Service de calendrier simulé initialisé avec un calendrier vide")
    
    def events(self):
        return self
    
    def list(self, calendarId='primary', timeMin=None, maxResults=10, singleEvents=True, orderBy=None):
        class MockResponse:
            def __init__(self, events):
                self.events = events
            
            def execute(self):
                return {"items": self.events}
        
        print(f"Simulation: Liste des événements (max {maxResults})")
        return MockResponse(self.events)
    
    def insert(self, calendarId='primary', body=None):
        class MockResponse:
            def __init__(self, event):
                self.event = event
            
            def execute(self):
                return self.event
        
        event = body.copy()
        event['id'] = str(self.next_id)
        self.next_id += 1
        self.events.append(event)
        print(f"Simulation: Événement créé - {event['summary']}")
        return MockResponse(event)

# Tenter d'obtenir le service Calendar
try:
    print("Initialisation du service Google Calendar...")
    calendar_service = get_calendar_service()
    print("✅ Configuration de Google Calendar réussie")
except Exception as e:
    print(f"❌ Erreur lors de la configuration de Google Calendar: {str(e)}")
    print("Passage en mode simulation pour la démonstration")
    USE_MOCK_CALENDAR = True
    calendar_service = MockCalendarService()

## 2. Création d'un serveur MCP pour Google Agenda

Maintenant, nous allons créer un serveur MCP qui expose des outils pour interagir avec Google Agenda. Ce serveur permettra à notre modèle de langage de:

1. Lister les événements à venir
2. Créer de nouveaux événements
3. Vérifier les disponibilités

Le serveur sera implémenté avec Flask et exposera une API REST conforme au Model Context Protocol.

In [ ]:
# Définition des outils MCP pour Google Agenda

CALENDAR_MCP_TOOLS = [
    {
        "name": "list_upcoming_events",
        "description": "Liste les événements à venir dans l'agenda",
        "parameters": {
            "type": "object",
            "properties": {
                "max_results": {
                    "type": "integer",
                    "description": "Nombre maximum d'événements à retourner (défaut: 10)"
                }
            },
            "required": []
        }
    },
    {
        "name": "create_event",
        "description": "Crée un nouvel événement dans l'agenda",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Titre de l'événement"
                },
                "description": {
                    "type": "string",
                    "description": "Description détaillée de l'événement"
                },
                "start_datetime": {
                    "type": "string",
                    "description": "Date et heure de début au format 'YYYY-MM-DD HH:MM'"
                },
                "end_datetime": {
                    "type": "string",
                    "description": "Date et heure de fin au format 'YYYY-MM-DD HH:MM'"
                },
                "location": {
                    "type": "string",
                    "description": "Lieu de l'événement (optionnel)"
                },
                "attendees": {
                    "type": "string",
                    "description": "Liste d'emails des participants, séparés par des virgules (optionnel)"
                }
            },
            "required": ["summary", "start_datetime", "end_datetime"]
        }
    },
    {
        "name": "check_availability",
        "description": "Vérifie les disponibilités pour une plage horaire donnée",
        "parameters": {
            "type": "object",
            "properties": {
                "start_datetime": {
                    "type": "string",
                    "description": "Date et heure de début au format 'YYYY-MM-DD HH:MM'"
                },
                "end_datetime": {
                    "type": "string",
                    "description": "Date et heure de fin au format 'YYYY-MM-DD HH:MM'"
                }
            },
            "required": ["start_datetime", "end_datetime"]
        }
    }
]

# Fonctions d'implémentation des outils
def list_upcoming_events(max_results=10):
    """Liste les événements à venir dans l'agenda."""
    try:
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indique l'heure UTC
        
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=now,
            maxResults=max_results,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            return {"message": "Aucun événement à venir trouvé."}
        
        formatted_events = []
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            
            # Formater la date pour une meilleure lisibilité
            if 'T' in start:  # Format datetime
                start_dt = datetime.datetime.fromisoformat(start.replace('Z', '+00:00'))
                start_formatted = start_dt.strftime('%Y-%m-%d %H:%M')
            else:  # Format date
                start_formatted = start
            
            formatted_events.append({
                "summary": event['summary'],
                "start": start_formatted,
                "id": event['id']
            })
        
        return {"events": formatted_events}
    
    except Exception as e:
        return {"error": f"Erreur lors de la récupération des événements: {str(e)}"}

def create_event(summary, start_datetime, end_datetime, description="", location="", attendees=""):
    """Crée un nouvel événement dans l'agenda."""
    try:
        # Convertir les dates au format ISO
        start_dt = datetime.datetime.strptime(start_datetime, '%Y-%m-%d %H:%M')
        end_dt = datetime.datetime.strptime(end_datetime, '%Y-%m-%d %H:%M')
        
        # Créer l'événement
        event = {
            'summary': summary,
            'location': location,
            'description': description,
            'start': {
                'dateTime': start_dt.isoformat(),
                'timeZone': 'Europe/Paris',
            },
            'end': {
                'dateTime': end_dt.isoformat(),
                'timeZone': 'Europe/Paris',
            }
        }
        
        # Ajouter les participants si spécifiés
        if attendees:
            event['attendees'] = [{'email': email.strip()} for email in attendees.split(',')]
        
        # Insérer l'événement dans le calendrier
        event = calendar_service.events().insert(calendarId='primary', body=event).execute()
        
        return {
            "message": f"Événement créé avec succès",
            "event_id": event.get('id'),
            "event_link": event.get('htmlLink', '')
        }
    
    except Exception as e:
        return {"error": f"Erreur lors de la création de l'événement: {str(e)}"}

def check_availability(start_datetime, end_datetime):
    """Vérifie les disponibilités pour une plage horaire donnée."""
    try:
        # Convertir les dates au format ISO
        start_dt = datetime.datetime.strptime(start_datetime, '%Y-%m-%d %H:%M')
        end_dt = datetime.datetime.strptime(end_datetime, '%Y-%m-%d %H:%M')
        
        # Récupérer les événements pour cette période
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start_dt.isoformat() + 'Z',
            timeMax=end_dt.isoformat() + 'Z',
            singleEvents=True
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            return {
                "available": True,
                "message": "La plage horaire est disponible."
            }
        else:
            return {
                "available": False,
                "message": f"La plage horaire n'est pas disponible. {len(events)} événement(s) trouvé(s).",
                "conflicting_events": [event['summary'] for event in events]
            }
    
    except Exception as e:
        return {"error": f"Erreur lors de la vérification des disponibilités: {str(e)}"}

In [ ]:
# Implémentation du serveur MCP pour Google Agenda avec Flask

from flask import Flask, request, jsonify
import threading
import time
import requests 
import json     

app = Flask(__name__)

@app.route('/tools', methods=['GET'])
def get_tools():
    return jsonify(CALENDAR_MCP_TOOLS)

@app.route('/run_tool', methods=['POST'])
def run_tool():
    data = request.json
    tool_name = data.get('name')
    parameters = data.get('parameters', {})
    
    if tool_name == "list_upcoming_events":
        max_results = parameters.get('max_results', 10)
        result = list_upcoming_events(max_results)
        return jsonify(result)
    
    elif tool_name == "create_event":
        summary = parameters.get('summary')
        start_datetime = parameters.get('start_datetime')
        end_datetime = parameters.get('end_datetime')
        description = parameters.get('description', '')
        location = parameters.get('location', '')
        attendees = parameters.get('attendees', '')
        
        if not all([summary, start_datetime, end_datetime]):
            return jsonify({"error": "Les paramètres summary, start_datetime et end_datetime sont requis"}), 400
        
        result = create_event(summary, start_datetime, end_datetime, description, location, attendees)
        return jsonify(result)
    
    elif tool_name == "check_availability":
        start_datetime = parameters.get('start_datetime')
        end_datetime = parameters.get('end_datetime')
        
        if not all([start_datetime, end_datetime]):
            return jsonify({"error": "Les paramètres start_datetime et end_datetime sont requis"}), 400
        
        result = check_availability(start_datetime, end_datetime)
        return jsonify(result)
    
    else:
        return jsonify({"error": "Outil non reconnu"}), 400

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "ok"})

# Fonction pour démarrer le serveur dans un thread séparé
def start_server():
    app.run(port=5001, debug=False)

# Démarrer le serveur dans un thread séparé
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True  # Le thread s'arrêtera quand le programme principal s'arrête
server_thread.start()

# Attendre que le serveur démarre
time.sleep(1)

# Vérifier que le serveur est en cours d'exécution
try:
    response = requests.get("http://localhost:5001/health")
    if response.status_code == 200:
        print("✅ Serveur MCP pour Google Agenda démarré avec succès sur le port 5001")
    else:
        print(f"❌ Le serveur a démarré mais renvoie un code d'état {response.status_code}")
except Exception as e:
    print(f"❌ Erreur lors du démarrage du serveur MCP: {str(e)}")

## 3. Test du serveur MCP pour Google Agenda

Maintenant que notre serveur MCP est en cours d'exécution, testons les différents outils qu'il expose.

In [ ]:
# Client MCP pour tester le serveur Google Agenda

class CalendarMCPClient:
    def __init__(self, server_url="http://localhost:5001"):
        self.server_url = server_url
    
    def get_tools(self):
        """Récupère la liste des outils disponibles."""
        try:
            response = requests.get(f"{self.server_url}/tools")
            if response.status_code == 200:
                return response.json()
            else:
                return f"Erreur: {response.status_code}"
        except Exception as e:
            return f"Erreur de connexion: {str(e)}"
    
    def run_tool(self, tool_name, parameters={}):
        """Exécute un outil sur le serveur MCP."""
        try:
            data = {
                "name": tool_name,
                "parameters": parameters
            }
            response = requests.post(f"{self.server_url}/run_tool", json=data)
            if response.status_code == 200:
                return response.json()
            else:
                return f"Erreur {response.status_code}: {response.text}"
        except Exception as e:
            return f"Erreur de connexion: {str(e)}"

# Créer une instance du client
calendar_client = CalendarMCPClient()

# Test 1: Récupérer la liste des outils
print("Test 1: Récupération des outils disponibles")
tools = calendar_client.get_tools()
print(json.dumps(tools, indent=2))

# Test 2: Lister les événements à venir
print("\nTest 2: Liste des événements à venir")
upcoming_events = calendar_client.run_tool("list_upcoming_events", {"max_results": 5})
print(json.dumps(upcoming_events, indent=2))

# Test 3: Vérifier la disponibilité pour demain
tomorrow = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
print(f"\nTest 3: Vérification de la disponibilité pour demain ({tomorrow})")
availability = calendar_client.run_tool("check_availability", {
    "start_datetime": f"{tomorrow} 10:00",
    "end_datetime": f"{tomorrow} 11:00"
})
print(json.dumps(availability, indent=2))

# Test 4: Créer un événement pour demain
print(f"\nTest 4: Création d'un événement pour demain ({tomorrow})")
event_result = calendar_client.run_tool("create_event", {
    "summary": "Réunion de test via MCP",
    "description": "Ceci est un test de création d'événement via le serveur MCP",
    "start_datetime": f"{tomorrow} 15:00",
    "end_datetime": f"{tomorrow} 16:00",
    "location": "Salle virtuelle",
    "attendees": ""
})
print(json.dumps(event_result, indent=2))

# Afficher la date actuelle pour vérification
print(f"\nDate actuelle: {datetime.datetime.now().strftime('%Y-%m-%d')}")
print(f"Date de demain: {tomorrow}")

## 4. Création d'un assistant de planification avec notre serveur MCP

Maintenant que notre serveur MCP fonctionne correctement, créons un assistant de planification qui utilise ces outils pour aider à gérer un agenda. Nous allons intégrer notre serveur MCP avec un modèle de langage (local ou OpenAI).

In [ ]:
# Configuration de la clé API OpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-*******"  # Remplacez par votre clé API
print("✅ Clé API OpenAI configurée")

In [ ]:
# Assistant de planification utilisant notre serveur MCP

from openai import OpenAI
import os

# Configuration pour le modèle de langage
# Choisir entre le modèle local ou OpenAI
USE_OPENAI = True  # Mettre à True pour utiliser OpenAI

# Configuration pour OpenAI
if USE_OPENAI:
    # Vérifier si la clé API est configurée
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("❌ Clé API OpenAI non trouvée. Veuillez la configurer.")
        print("Exemple: os.environ['OPENAI_API_KEY'] = 'votre-clé-api'")
    else:
        print("✅ Clé API OpenAI configurée")
    
    # Créer le client OpenAI
    client = OpenAI(api_key=api_key)
else:
    # Configuration pour le modèle local
    try:
        client = OpenAI(
            base_url="http://localhost:8080/v1",
            api_key="sk-no-key-required"
        )
        print("✅ Client pour le modèle local configuré")
    except Exception as e:
        print(f"❌ Erreur lors de la configuration du client pour le modèle local: {str(e)}")

class PlanningAssistant:
    def __init__(self, use_openai=False):
        self.use_openai = use_openai
        self.client = client
        self.calendar_client = CalendarMCPClient()
        self.conversation_history = []
        
        # Récupérer les outils disponibles
        self.tools = self.calendar_client.get_tools()
        
        if isinstance(self.tools, str) and self.tools.startswith("Erreur"):
            print(f"❌ Impossible de récupérer les outils: {self.tools}")
            self.tools = []
    
    def _format_tools_for_openai(self):
        """Convertit les outils MCP au format OpenAI."""
        openai_tools = []
        for tool in self.tools:
            openai_tools.append({
                "type": "function",
                "function": {
                    "name": tool["name"],
                    "description": tool["description"],
                    "parameters": tool["parameters"]
                }
            })
        return openai_tools
    
    def process_query(self, user_query):
        """Traite une requête utilisateur et renvoie une réponse."""
        # Ajouter la requête à l'historique
        self.conversation_history.append({"role": "user", "content": user_query})
        
        if self.use_openai:
            return self._process_with_openai(user_query)
        else:
            return self._process_with_local_model(user_query)
    
    def _process_with_openai(self, user_query):
        """Traite la requête avec l'API OpenAI."""
        try:
            # Convertir les outils au format OpenAI
            openai_tools = self._format_tools_for_openai()
            
            # Appeler l'API avec les outils
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=self.conversation_history,
                tools=openai_tools,
                tool_choice="auto"
            )
            
            # Récupérer le message de réponse
            message = response.choices[0].message
            
            # Vérifier si le modèle veut utiliser un outil
            if message.tool_calls:
                # Exécuter chaque appel d'outil
                for tool_call in message.tool_calls:
                    # Extraire les informations de l'outil
                    tool_name = tool_call.function.name
                    tool_params = json.loads(tool_call.function.arguments)
                    
                    print(f"Le modèle utilise l'outil: {tool_name}")
                    
                    # Exécuter l'outil via le serveur MCP
                    tool_result = self.calendar_client.run_tool(tool_name, tool_params)
                    
                    # Ajouter l'appel d'outil à l'historique
                    self.conversation_history.append({
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [{
                            "id": tool_call.id,
                            "type": "function",
                            "function": {
                                "name": tool_name,
                                "arguments": tool_call.function.arguments
                            }
                        }]
                    })
                    
                    # Ajouter le résultat de l'outil à l'historique
                    self.conversation_history.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(tool_result)
                    })
                
                # Obtenir la réponse finale du modèle
                final_response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=self.conversation_history
                )
                
                final_content = final_response.choices[0].message.content
                self.conversation_history.append({"role": "assistant", "content": final_content})
                return final_content
            else:
                # Le modèle a répondu directement sans utiliser d'outil
                content = message.content
                self.conversation_history.append({"role": "assistant", "content": content})
                return content
        
        except Exception as e:
            error_msg = f"Erreur lors de l'appel à l'API: {str(e)}"
            print(error_msg)
            return error_msg
    
    def _process_with_local_model(self, user_query):
        """Traite la requête avec le modèle local."""
        try:
            # Construire un prompt qui explique les outils disponibles
            tools_description = ""
            for tool in self.tools:
                tools_description += f"- {tool['name']}: {tool['description']}\n"
                if 'parameters' in tool and 'properties' in tool['parameters']:
                    tools_description += "  Paramètres:\n"
                    for param_name, param_info in tool['parameters']['properties'].items():
                        param_desc = param_info.get('description', '')
                        tools_description += f"  - {param_name}: {param_desc}\n"
            
            # Construire le prompt avec l'historique de conversation
            conversation_prompt = ""
            for message in self.conversation_history:
                role = message["role"]
                content = message.get("content")
                if role == "user" and content:
                    conversation_prompt += f"Utilisateur: {content}\n\n"
                elif role == "assistant" and content:
                    conversation_prompt += f"Assistant: {content}\n\n"
            
            # Construire le prompt final avec instructions pour utiliser les outils
            system_prompt = f"""Tu es un assistant de planification qui peut gérer l'agenda Google d'un utilisateur.
            Tu peux créer des événements, vérifier les disponibilités et lister les événements à venir.
            
            Voici les outils disponibles:
            {tools_description}
            
            Pour utiliser un outil, réponds au format suivant:
            UTILISER_OUTIL: nom_de_l_outil PARAMÈTRES:

            param1: valeur1
            param2: valeur2
                        
            Si tu n'as pas besoin d'utiliser d'outil, réponds normalement.
            """
            
            full_prompt = f"{system_prompt}\n\n{conversation_prompt}Assistant: "
            
            try:
                # Appeler le modèle local
                response = self.client.chat.completions.create(
                    model="local-model",
                    messages=[{"role": "user", "content": full_prompt}],
                    max_tokens=1000
                )
                
                model_response = response.choices[0].message.content
            except Exception as e:
                print(f"Erreur avec l'API chat: {str(e)}")
                print("Tentative avec l'API de complétion...")
                
                # Repli sur l'API de complétion
                response = requests.post(
                    "http://localhost:8080/completion",
                    json={
                        "prompt": full_prompt,
                        "max_tokens": 1000,
                        "temperature": 0.7,
                        "stop": ["\nUtilisateur:", "\n\nUtilisateur:"]
                    }
                )
                
                if response.status_code != 200:
                    return f"Erreur {response.status_code} lors de l'appel au modèle local: {response.text}"
                
                model_response = response.json().get("content", "")
            
            # Analyser la réponse pour détecter un appel d'outil
            import re
            tool_call_match = re.search(r'UTILISER_OUTIL: (\w+)\s+PARAMÈTRES:((?:\s*- \w+: .+)+)', model_response, re.DOTALL)
            
            if tool_call_match:
                # Extraire le nom de l'outil et les paramètres
                tool_name = tool_call_match.group(1)
                params_text = tool_call_match.group(2)
                
                # Extraire les paramètres
                param_matches = re.findall(r'- (\w+): (.+)(?:\n|$)', params_text)
                tool_params = {name: value.strip() for name, value in param_matches}
                
                print(f"Le modèle utilise l'outil: {tool_name}")
                print(f"Avec les paramètres: {tool_params}")
                
                # Exécuter l'outil via le serveur MCP
                tool_result = self.calendar_client.run_tool(tool_name, tool_params)
                print(f"Résultat de l'outil: {json.dumps(tool_result)}")
                
                # Construire un nouveau prompt avec le résultat de l'outil
                tool_prompt = f"{full_prompt}Je vais utiliser l'outil {tool_name}.\n\nRésultat de l'outil: {json.dumps(tool_result, ensure_ascii=False)}\n\nMaintenant, je vais répondre à la question: "
                
                # Appeler à nouveau le modèle avec le résultat de l'outil
                try:
                    final_response = self.client.chat.completions.create(
                        model="local-model",
                        messages=[{"role": "user", "content": tool_prompt}],
                        max_tokens=1000
                    )
                    
                    final_content = final_response.choices[0].message.content
                except Exception as e:
                    print(f"Erreur avec l'API chat pour la réponse finale: {str(e)}")
                    
                    # Repli sur l'API de complétion
                    final_response = requests.post(
                        "http://localhost:8080/completion",
                        json={
                            "prompt": tool_prompt,
                            "max_tokens": 1000,
                            "temperature": 0.7,
                            "stop": ["\nUtilisateur:", "\n\nUtilisateur:"]
                        }
                    )
                    
                    if final_response.status_code != 200:
                        return f"Erreur {final_response.status_code} lors de l'appel au modèle local: {final_response.text}"
                    
                    final_content = final_response.json().get("content", "")
                
                self.conversation_history.append({"role": "assistant", "content": final_content})
                return final_content
            
            # Si aucun appel d'outil n'est détecté, retourner la réponse directe
            self.conversation_history.append({"role": "assistant", "content": model_response})
            return model_response
            
        except Exception as e:
            error_msg = f"Erreur lors du traitement avec le modèle local: {str(e)}"
            print(error_msg)
            return error_msg

# Créer l'assistant de planification
planning_assistant = PlanningAssistant(use_openai=USE_OPENAI)

## 5. Test de l'assistant de planification

Testons maintenant notre assistant de planification avec quelques requêtes pour voir comment il utilise les outils MCP pour gérer l'agenda.

In [ ]:
# Fonction pour afficher les conversations de manière plus lisible
def display_conversation(query, response):
    display(HTML(f"""
    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
        <strong>Vous:</strong> {query}
    </div>
    <div style="background-color: #e8f4f8; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <strong>Assistant:</strong> {response}
    </div>
    """))

# Obtenir la date de demain au format lisible
tomorrow_date = (datetime.datetime.now() + datetime.timedelta(days=1))
tomorrow = tomorrow_date.strftime('%Y-%m-%d')
tomorrow_readable = tomorrow_date.strftime('%d/%m/%Y')

# Test 1: Demander les événements à venir
query1 = "Quels sont mes prochains rendez-vous?"
print("Traitement de la requête: " + query1)
response1 = planning_assistant.process_query(query1)
display_conversation(query1, response1)

# Test 2: Vérifier la disponibilité
query2 = f"Suis-je disponible demain ({tomorrow_readable}) entre 14h et 16h?"
print("Traitement de la requête: " + query2)
response2 = planning_assistant.process_query(query2)
display_conversation(query2, response2)

# Test 3: Créer un événement
query3 = f"Crée une réunion intitulée 'Point d'équipe hebdomadaire' demain ({tomorrow_readable}) de 10h à 11h avec la description 'Discussion des avancées de la semaine'"
print("Traitement de la requête: " + query3)
response3 = planning_assistant.process_query(query3)
display_conversation(query3, response3)

## 6. Interface interactive pour l'assistant de planification

Créons maintenant une interface interactive pour interagir avec notre assistant de planification.

In [ ]:
# Interface interactive avec ipywidgets
from ipywidgets import widgets
from IPython.display import display, clear_output

# Installer ipywidgets si nécessaire
!pip install -q ipywidgets

# Créer les widgets
input_box = widgets.Textarea(
    value='',
    placeholder='Posez votre question sur votre agenda...',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

output_area = widgets.Output(layout=widgets.Layout(width='100%', border='1px solid black', padding='10px'))

use_openai_checkbox = widgets.Checkbox(
    value=USE_OPENAI,
    description='Utiliser OpenAI API',
    disabled=not os.getenv("OPENAI_API_KEY")
)

submit_button = widgets.Button(
    description='Envoyer',
    button_style='primary',
    tooltip='Envoyer votre question'
)

reset_button = widgets.Button(
    description='Nouvelle conversation',
    button_style='warning',
    tooltip='Démarrer une nouvelle conversation'
)

# Variable pour stocker l'assistant
current_assistant = None

# Fonction pour gérer le clic sur le bouton d'envoi
def on_submit_button_clicked(b):
    global current_assistant
    
    # Créer l'assistant si nécessaire
    if current_assistant is None:
        try:
            use_openai = use_openai_checkbox.value
            current_assistant = PlanningAssistant(use_openai=use_openai)
            with output_area:
                clear_output()
                if use_openai:
                    print("✅ Assistant initialisé avec l'API OpenAI")
                else:
                    print("✅ Assistant initialisé avec le modèle local")
        except Exception as e:
            with output_area:
                clear_output()
                print(f"❌ Erreur lors de la création de l'assistant: {str(e)}")
            return
    
    # Récupérer la question
    query = input_box.value
    if not query.strip():
        return
    
    # Effacer la zone de saisie
    input_box.value = ''
    
    # Afficher la question et obtenir la réponse
    with output_area:
        print(f"Vous: {query}")
        print("\nAssistant: Réflexion en cours...")
        
        try:
            # Obtenir la réponse
            response = current_assistant.process_query(query)
            
            # Afficher la réponse
            clear_output()
            print(f"Vous: {query}")
            print(f"\nAssistant: {response}")
        except Exception as e:
            clear_output()
            print(f"Vous: {query}")
            print(f"\n❌ Erreur: {str(e)}")

# Fonction pour gérer le clic sur le bouton de réinitialisation
def on_reset_button_clicked(b):
    global current_assistant
    current_assistant = None
    with output_area:
        clear_output()
        print("✅ Nouvelle conversation démarrée.")

# Attacher les fonctions aux boutons
submit_button.on_click(on_submit_button_clicked)
reset_button.on_click(on_reset_button_clicked)

# Afficher l'interface
display(widgets.HTML("<h3>Assistant de Planification Google Agenda</h3>"))
display(widgets.HBox([use_openai_checkbox]))
display(input_box)
display(widgets.HBox([submit_button, reset_button]))
display(output_area)

# Message initial
with output_area:
    print("Assistant prêt. Posez votre question sur votre agenda!")

## 7. Exemples d'utilisation de l'assistant de planification

Voici quelques exemples de questions que vous pouvez poser à votre assistant de planification:

1. **Lister les événements**
   - "Quels sont mes rendez-vous pour cette semaine?"
   - "Montre-moi mes 3 prochains événements"
   - "Ai-je des réunions prévues aujourd'hui?"

2. **Vérifier les disponibilités**
   - "Suis-je disponible demain entre 14h et 16h?"
   - "Est-ce que j'ai du temps libre vendredi après-midi?"
   - "Quand suis-je disponible cette semaine pour une réunion de 2 heures?"

3. **Créer des événements**
   - "Crée une réunion intitulée 'Point d'équipe' demain de 10h à 11h"
   - "Ajoute un rendez-vous médical le 15 du mois prochain à 9h30 pour 1 heure"
   - "Planifie une visioconférence avec l'équipe marketing jeudi à 15h avec comme participants jean@example.com et marie@example.com"

4. **Requêtes complexes**
   - "Trouve un créneau de 2 heures disponible cette semaine et crée une réunion intitulée 'Préparation présentation'"
   - "Quels sont mes rendez-vous de demain et ai-je le temps d'ajouter une réunion d'une heure?"

## 8. Nettoyage et arrêt du serveur

Avant de terminer, arrêtons proprement le serveur MCP que nous avons lancé.

In [ ]:
# Arrêter le serveur MCP
import os
import signal
import sys
import requests

def stop_server(port=5001):
    """Arrête le serveur Flask en cours d'exécution sur le port spécifié."""
    try:
        # Vérifier si le serveur est toujours en cours d'exécution
        try:
            requests.get(f"http://localhost:{port}/health", timeout=1)
            server_running = True
        except:
            server_running = False
        
        if not server_running:
            print(f"Aucun serveur en cours d'exécution sur le port {port}")
            return
        
        # Trouver et tuer le processus
        if sys.platform.startswith('win'):
            # Windows
            os.system(f"FOR /F \"tokens=5\" %P IN ('netstat -ano ^| findstr :{port}') DO taskkill /F /PID %P")
        else:
            # Linux/Mac
            os.system(f"kill $(lsof -t -i:{port})")
        
        print(f"Serveur sur le port {port} arrêté avec succès")
    except Exception as e:
        print(f"Erreur lors de l'arrêt du serveur: {str(e)}")

# Arrêter le serveur MCP
stop_server(5001)

## Conclusion

Dans ce notebook, nous avons:

1. **Créé un serveur MCP pour Google Agenda**
   - Implémentation d'outils pour lister les événements, vérifier les disponibilités et créer des événements
   - Configuration de l'authentification OAuth pour accéder à l'API Google Calendar

2. **Testé le serveur MCP avec un client simple**
   - Vérification du bon fonctionnement des différents outils
   - Manipulation des événements de l'agenda

3. **Créé un assistant de planification intelligent**
   - Intégration du serveur MCP avec un modèle de langage (local ou OpenAI)
   - Traitement des requêtes en langage naturel pour gérer un agenda

4. **Développé une interface interactive**
   - Interface utilisateur simple pour interagir avec l'assistant
   - Gestion des conversations et des erreurs

Cette approche montre comment les agents IA peuvent interagir avec des services externes comme Google Agenda grâce au Model Context Protocol, offrant ainsi des assistants intelligents capables d'effectuer des actions concrètes dans le monde réel.